In [14]:
import yaml
import os
# !pip install e3nn
from e3nn import o3

In [15]:
def load_config(config_file_path):
    with open(config_file_path, 'r') as stream:
        try:
            config = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return config

def save_config(config, config_file_path):
    with open(config_file_path, 'w') as stream:
        try:
            yaml.dump(config, stream, default_flow_style=False)
        except yaml.YAMLError as exc:
            print(exc)

def update_config(config, **kwargs):
    for key, value in kwargs.items():
        config[key] = value
    return config

In [18]:
example_config_file_path="old_configs/aspirin.yaml"
num_features=32
max_epochs=200
root="results/aspirin"
out_dir_configs="aspirin_configs"

!rm -rf {out_dir_configs}
!mkdir {out_dir_configs}

n_val=50
for n_train in [150,550,950]:
    for conv0_layers in range(3):
        for lmax in range(4):

            config = load_config(example_config_file_path)

            num_layers=4-conv0_layers
            feature_irreps_hidden=str(o3.Irreps(
                            [
                                (num_features, (l, p))
                                for p in ((1, -1) if True else (1,))
                                for l in range(lmax + 1)
                            ]
                        ))
            run_name=f"ntrain_{n_train}_lmax_{lmax}_invariant_{conv0_layers}_epochs_{max_epochs}"

            config = update_config(config, n_val=n_val,num_layers=num_layers, feature_irreps_hidden=feature_irreps_hidden, max_epochs=max_epochs, root=root, run_name=run_name, conv0_layers=conv0_layers, n_train=n_train)
            save_config(config, f"{out_dir_configs}/{run_name}.yaml")